In [40]:
%run PilotForestAreas.ipynb
%run CarbonRevenue.ipynb

In [41]:
import param as pm
import panel as pn
import pandas as pd
import hvplot.pandas
pn.extension()

In [62]:
class ForestProject(pm.Parameterized):
    month = pm.Integer(-1, bounds=(-1,None), doc="""
        Represents the number of months in the future that this land is set to be acquired and centree initiated.
        -1 means not scheduled. 0 means starting this month. Positive integer is number of months in future.""")
    forest = pm.ClassSelector(class_=Forest)
    carbon = pm.ClassSelector(class_=Carbon)
    
    def compute(self):
        return self.carbon.compute(self.forest)
    
    def view_table(self):
        return self.compute().reset_index().hvplot.table()

In [74]:
class ForestPipeline(pm.Parameterized):
    projects = pm.List()
    
        
    def __init__(self, projects: list):
        self.projects = [ForestProject(**data) for data in projects]
        
    def view_pipeline(self):
        return pn.Tabs(*[(p.forest.forest_name, p.view_table()) for p in self.projects])
    
    def export_formatted(self):
        df = pd.concat([p.forest.export() for p in self.projects]).T.applymap(Forest._format_numeric)
        return df
    
    def view_export_formatted(self):
        return self.export_formatted().reset_index().hvplot.table()

In [77]:
carbon_model = Carbon(below_soil_tonnage_factor=0, carbon_price_appreciation=0.15)

forest_project_data = [
    {'month': 8, 'forest': f1, 'carbon': carbon_model},
    {'month': 15, 'forest': f2, 'carbon': carbon_model},
]

In [78]:
forest_pipeline = ForestPipeline(forest_project_data)

In [80]:
forest_pipeline.view_pipeline()

Tabs
    [0] HoloViews(Table, height=300, name='KNF1', width=700)
    [1] HoloViews(Table, height=300, name='KNF2', width=700)

In [100]:
forest_pipeline.view_export_formatted()

:Table   [Field,Type,('KNF1',),('KNF2',)]